# Import Necessary Modules

In [0]:
from pyspark.sql.functions import trim, col, lit, when, try_to_date, date_format, count, date_add, round, split, date_sub

# Read Data

In [0]:
df_silver = spark.read.format("parquet") \
    .load("abfss://bronze@globalsalestorage.dfs.core.windows.net/Raw_Data")

In [0]:
df_silver.display()

# Schema validation

## Convert DataType - Date Columns


In [0]:
# Trim leading and trailing spaces from Order_Date and Ship_Date columns
df_silver = df_silver.withColumn("Order_Date", trim(col("Order_Date"))) \
                     .withColumn("Ship_Date", trim(col("Ship_Date")))

# Convert Order_Date and Ship_Date columns to date format
df_silver = df_silver.withColumn("Order_Date", try_to_date(col("Order_Date"), "yyyy-MM-dd")) \
                     .withColumn("Ship_Date", try_to_date(col("Ship_Date"), "yyyy-MM-dd"))


# Check for Nulls / Missing Values

**Check Order_ID Valide - If Invalide Write Date to RejectData Condainer**

In [0]:
invalid_order_id = df_silver.filter(col("Order_ID").isNull())  # Filter rows with null Order_ID

if invalid_order_id.count() > 0:
    invalid_order_id.write\
        .format("delta")\
        .mode("append")\
        .save("abfss://rejectdata@globalsalestorage.dfs.core.windows.net/e_commerce/invalid_order_id")

In [0]:
df_silver = df_silver.dropDuplicates(["Order_ID"]) # Remove duplicate rows based on Order_ID
df_silver = df_silver.filter(col("Order_ID").isNotNull())  # Remove rows with null Order_ID

**Get Order_ID & Customer_ID & Product_ID**

In [0]:
df_silver=df_silver.withColumn("Order_ID",split(col("Order_ID"),"-").getItem(0))
df_silver = df_silver.withColumn("Customer_ID",split(col("Customer_ID"),"-").getItem(0))
df_silver = df_silver.withColumn("Product_ID",split(col("Product_ID"),"-").getItem(0))

**Check Customer_ID Valide - If Invalide Write Date to RejectData Condainer**

In [0]:
invalid_customer_id = df_silver.filter(col("Customer_ID").isNull())  # Filter rows with null Customer_ID

if invalid_customer_id.count() > 0:
    invalid_customer_id.write\
        .format("delta")\
        .mode("append")\
        .save("abfss://rejectdata@globalsalestorage.dfs.core.windows.net/e_commerce/invalid_customer_id")

In [0]:
df_silver = df_silver.filter(col("Customer_ID").isNotNull())  # Remove rows with null Order_ID

**Check Order_Date & Ship_Date Valide - If Invalide Write Date to RejectData Condainer**

In [0]:
df_reject = df_silver.filter(col("Order_Date").isNull() & col("Ship_Date").isNull())

if df_reject.count() > 0:
    df_reject.write\
        .format("delta")\
        .mode("append")\
        .save("abfss://rejectdata@globalsalestorage.dfs.core.windows.net/e_commerce/invalid_date")



In [0]:
df_silver = df_silver.filter(col("Order_Date").isNotNull() & col("Ship_Date").isNotNull())


In [0]:
null_count_od = df_silver.filter(col("Order_Date").isNull()).count()
null_count_sd = df_silver.filter(col("Ship_Date").isNull()).count()

if null_count_od > 0 or null_count_sd > 0:
    df_silver = df_silver.withColumn(
        "Order_Date",
        when(col("Order_Date").isNull() & col("Ship_Date").isNotNull(), date_sub(col("Ship_Date"), 2))
        .otherwise(col("Order_Date"))
    ).withColumn(
        "Ship_Date",
        when(col("Ship_Date").isNull() & col("Order_Date").isNotNull(), date_add(col("Order_Date"), 2))
        .otherwise(col("Ship_Date"))
    )

In [0]:
count_invalid_dates = df_silver.filter(col("Order_Date") > col("Ship_Date")).count()

if count_invalid_dates > 0:
    df_silver = df_silver.withColumn(
        "Ship_Date",
        when(col("Order_Date") > col("Ship_Date"), date_add(col("Order_Date"), 2))
        .otherwise(col("Ship_Date"))
    )

#Strip spaces from all string columns

In [0]:
# Get all string columns
string_cols = [c for c, t in df_silver.dtypes if t == "string"]

# Apply trim to all string columns
for c in string_cols:
    df_silver = df_silver.withColumn(c, trim(col(c)))

## Replace Missing value with N/A

In [0]:
# Replace null or empty values with "N/A" for string columns
df_silver = df_silver.withColumn(
    "Customer_Name",
    when(col("Customer_Name").isNull() | (col("Customer_Name") == ""), lit("N/A"))
    .otherwise(col("Customer_Name"))
)

df_silver = df_silver.withColumn(
    "Segment",
    when(col("Segment").isNull() | (col("Segment") == ""), lit("N/A"))
    .otherwise(col("Segment"))
)

df_silver = df_silver.withColumn(
    "City",
    when(col("City").isNull() | (col("City") == ""), lit("N/A"))
    .otherwise(col("City"))
)

df_silver = df_silver.withColumn(
    "State",
    when(col("State").isNull() | (col("State") == ""), lit("N/A"))
    .otherwise(col("State"))
)

df_silver = df_silver.withColumn(
    "Country",
    when(col("Country").isNull() | (col("Country") == ""), lit("N/A"))
    .otherwise(col("Country"))
)

df_silver = df_silver.withColumn(
    "Region",
    when(col("Region").isNull() | (col("Region") == ""), lit("N/A"))
    .otherwise(col("Region"))
)

df_silver = df_silver.withColumn(
    "Product_Name",
    when(col("Product_Name").isNull() | (col("Product_Name") == ""), lit("N/A"))
    .otherwise(col("Product_Name"))
)

df_silver = df_silver.withColumn(
    "Category",
    when(col("Category").isNull() | (col("Category") == ""), lit("N/A"))
    .otherwise(col("Category"))
)

df_silver = df_silver.withColumn(
    "Sub_Category",
    when(col("Sub_Category").isNull() | (col("Sub_Category") == ""), lit("N/A"))
    .otherwise(col("Sub_Category"))
)

df_silver = df_silver.withColumn(
    "Order_Priority",
    when(col("Order_Priority").isNull() | (col("Order_Priority") == ""), lit("N/A"))
    .otherwise(col("Order_Priority"))
)

# Clean Numeric Columns (sales,quantity,discount,profit)

In [0]:
from pyspark.sql.functions import col, when, lit, abs, bround

df_silver = df_silver.withColumn(
    "Quantity",
    when(col("Quantity").isNull(), lit(0)).otherwise(abs(col("Quantity")))
)

df_silver = df_silver.withColumn(
    "Sales",
    when(col("Sales").isNull(), lit(0)).otherwise(abs(col("Sales")))
)

df_silver = df_silver.withColumn(
    "Shipping_Cost",
    when(col("Shipping_Cost").isNull() | (col("Shipping_Cost") < 0), lit(0)).otherwise(col("Shipping_Cost"))
)

df_silver = df_silver.withColumn(
    "Discount",
    when(col("Discount").isNull() | (col("Discount") < 0) | (col("Discount") > 1), lit(0)).otherwise(col("Discount"))
)

df_silver = df_silver.withColumn(
    "Profit",
    col("Sales") * (1 - col("Discount"))
)

df_silver = df_silver.withColumn(
    "price",
    bround(when(col("Quantity") == 0, lit(0)).otherwise(col("Sales") / col("Quantity")), 2 )
)

# Write Date to Silver Layer

In [0]:
df_silver.write.format("parquet").mode("overwrite").save("abfss://silver@globalsalestorage.dfs.core.windows.net/clean_e_commerce_data")